In [ ]:
{
    "serves": [
        {
            "type": "First Serve",
            "sides": {
                "deuce": {
                    "wide": 20,
                    "body": 30,
                    "t": 50
                },
                "ad": {
                    "wide": 10,
                    "body": 20,
                    "t": 10
                }
            }
        },
        {
            "type": "Second Serve",
            "sides": {
                "deuce": {
                    "wide": 25,
                    "body": 30,
                    "t": 50
                },
                "ad": {
                    "wide": 15,
                    "body": 20,
                    "t": 10
                }
            }
        }
    ]
}

In [25]:
import pandas as pd
import json

def label_zone(x):
    # wide, body, or t
    if x == '' or pd.isnull(x):
        return ''
    x = float(x)
    if x < -105 or x > 105:
        return 'wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        return 'body'
    elif -52.5 < x < 52.5:
        return 't'
    return ''

def scaled_x(stroke, x):
    # Scale x-coordinate for first or second serve
    if pd.isnull(x):
        return None
    if stroke == 'first_serve':
        return float(x) * 38.2764654418
    if stroke == 'second_serve':
        return float(x) * 38.2764654418
    return None

def extract_rudy_quan_serves(player_name):
    # Load file 
    player_name = 'Rudy Quan'
    file_path = f'../../data/mens/{player_name}/combined.xlsx'
    df = pd.read_excel(file_path, sheet_name='Shots')
    # Only keep serves by Rudy Quan
    serves = df[(df['player'] == 'Rudy Quan') & (df['stroke'].str.contains('serve', case=False, na=False))]
    # Check if 'side' column exists
    has_side = 'side' in serves.columns
    output = {"serves": []}
    for serve_type, serve_label in [('first_serve', 'First Serve'), ('second_serve', 'Second Serve')]:
        # Serve type
        serve_df = serves[serves['stroke'].str.lower() == serve_type]
        counts = {
            "deuce": {"wide": 0, "body": 0, "t": 0},
            "ad": {"wide": 0, "body": 0, "t": 0}
        }
        for idx, row in serve_df.iterrows():

            side = 'deuce'
            if has_side:
                val = str(row['side']).strip().lower()
                if val.startswith('a'):
                    side = 'ad'
                elif val.startswith('d'):
                    side = 'deuce'
            x = row.get('Bounce(x)', '')
            x_val = scaled_x(serve_type, x)
            zone = label_zone(x_val)
            if zone in ['wide', 'body', 't']:
                counts[side][zone] += 1
        output["serves"].append({
            "type": serve_label,
            "sides": counts
        })
    return json.dumps(output, indent=4)


